Copyright 2024 Google. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
import pandas as pd
import jax.numpy as jnp
import jax
import scipy
import numpy as np

In [ ]:
data = pd.read_csv('answers.csv')
data

,methodA,methodB,originalName,answerValue
0,mozjpeg-q80-yuv422,jpegli-q70-yuv444,1044329,B
1,mozjpeg-q70-yuv420,libjpeg-turbo-q75-yuv420,792079,A
2,mozjpeg-q95-yuv444,jpegli-q90-yuv444,1420710,B
3,mozjpeg-q85-yuv422,libjpeg-turbo-q80-yuv422,pexels-photo-4210863,B
4,mozjpeg-q70-yuv420,jpegli-q55-yuv444,6292444,B
...,...,...,...,...
13483,mozjpeg-q70-yuv420,jpegli-q90-yuv444,3762075,A
13484,libjpeg-turbo-q80-yuv422,jpegli-q85-yuv444,pexels-photo-2686358,A
13485,mozjpeg-q85-yuv422,jpegli-q75-yuv444,2670327,B
13486,jpegli-q85-yuv444,jpegli-q80-yuv444,ularapi_Semarang_City_Logo,B


In [ ]:
methods = np.unique(np.concatenate([data['methodA'].unique(), data['methodB'].unique()]))
initial_elo = 2000
elo_n = 400
elos = jnp.ones((len(methods), )) * initial_elo

win_count_list = []
for method_a in methods:
  method_win_list = []
  for method_b in methods:
    if method_a == method_b:
      method_win_list.append(0)
    else:
      count = data[(data['methodA'] == method_a) & (data['methodB'] == method_b) & (data['answerValue'] == 'B')].shape[0]
      count += data[(data['methodA'] == method_b) & (data['methodB'] == method_a) & (data['answerValue'] == 'A')].shape[0]
      method_win_list.append(count)
  win_count_list.append(method_win_list)
win_count = jnp.asarray(win_count_list)

def log_win_prob(elo_a, elo_b):
  scaled_elo_a = elo_a * jnp.log(10) / elo_n
  scaled_elo_b = elo_b * jnp.log(10) / elo_n
  return jax.nn.log_sigmoid(scaled_elo_a - scaled_elo_b)

def loss(elos):
  shifted_elos = elos - initial_elo
  regularizer_factor = 1e-5 / elos.shape[0]
  sum_regularizer = 0.9 / elos.shape[0]
  accum = -0.5 * regularizer_factor * jnp.square(shifted_elos).sum() - 0.5 * sum_regularizer * jnp.square(shifted_elos.sum())
  log_probs = log_win_prob(elos[:, jnp.newaxis], elos[jnp.newaxis, :])
  log_probs_scaled = win_count * log_probs
  return -(accum + log_probs_scaled.sum())

loss_jit = jax.jit(loss)
loss_jit_grad = jax.grad(loss_jit)

result = scipy.optimize.minimize(loss_jit, elos, jac=loss_jit_grad, method='BFGS')
print(f'{result=}')
method_scores = list(zip(methods, result.x))
method_scores.sort(key=lambda x: x[1])
method_scores

result=  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 7355.83447265625
        x: [ 1.702e+03  1.747e+03 ...  2.267e+03  2.448e+03]
      nit: 37
      jac: [ 4.124e-04  1.745e-03 ...  1.090e-03 -4.406e-04]
 hess_inv: [[ 4.059e+01  3.444e+01 ... -3.696e+01 -3.968e+01]
            [ 3.444e+01  3.155e+01 ... -3.387e+01 -3.607e+01]
            ...
            [-3.696e+01 -3.387e+01 ...  4.109e+01  4.488e+01]
            [-3.968e+01 -3.607e+01 ...  4.488e+01  1.075e+02]]
     nfev: 97
     njev: 86


[('libjpeg-turbo-q55-yuv420', 1544.1666960675595),
 ('libjpeg-turbo-q60-yuv420', 1637.898323179558),
 ('libjpeg-turbo-q65-yuv420', 1673.7722908631165),
 ('jpegli-q65-yuv420', 1697.62945079494),
 ('jpegli-q55-yuv444', 1702.3819453028395),
 ('mozjpeg-q70-yuv420', 1739.4859576416322),
 ('jpegli-q60-yuv444', 1746.5323602503968),
 ('libjpeg-turbo-q70-yuv420', 1764.6265441942764),
 ('jpegli-q70-yuv420', 1774.358484610835),
 ('jpegli-q65-yuv444', 1805.7287795052687),
 ('mozjpeg-q75-yuv420', 1822.4528151830204),
 ('libjpeg-turbo-q75-yuv420', 1831.2530373401673),
 ('jpegli-q75-yuv420', 1872.6390413478834),
 ('jpegli-q70-yuv444', 1898.3800995471095),
 ('mozjpeg-q80-yuv422', 1963.831148876465),
 ('jpegli-q80-yuv420', 1988.8932000963903),
 ('libjpeg-turbo-q80-yuv422', 2000.2437132771747),
 ('jpegli-q75-yuv444', 2023.39019887469),
 ('jpegli-q85-yuv420', 2104.1339364762284),
 ('mozjpeg-q85-yuv422', 2107.951461222033),
 ('jpegli-q80-yuv444', 2108.894131436216),
 ('libjpeg-turbo-q85-yuv422', 2114.5248